In [3]:
from pathlib import Path
from synse.helpers import pmb_generator
from synse.sbn import SBNGraph, SBN_NODE_TYPE

multi_boxes = []
for path in pmb_generator('../../data/pmb_dataset/pmb-extracted/pmb-4.0.0/data/en/gold/p01/', "**/*.sbn"):
    G = SBNGraph().from_path(path)
    b_nodes = len([b for b in G.nodes if b[0] == SBN_NODE_TYPE.BOX])
    if b_nodes > 1:
        multi_boxes.append(path)

ModuleNotFoundError: No module named 'synse'

In [8]:
len(multi_boxes)

47